In [1]:
import mlflow

In [2]:
import numpy as np
import glob
import os
import keras
from keras.utils import to_categorical

# Asumiendo que los archivos son .txt y que el directorio está en el path actual
file_paths = glob.glob("./UCIHARDataset/train/InertialSignals/*.txt")

# Lista para almacenar los datos de cada archivo
data_list = []

for file in file_paths:
    # Carga los datos del archivo en un array de numpy
    data = np.loadtxt(file)
    data_list.append(data)
    

# Convierte la lista de arrays en un solo array de numpy.
# Nota: Esto funcionará si todos los arrays tienen la misma forma.
data_array = np.array(data_list)
x_train = data_array.transpose(1, 2, 0)

file_paths = glob.glob("./UCIHARDataset/test/InertialSignals/*.txt")
# Lista para almacenar los datos de cada archivo
data_list = []

for file in file_paths:
    # Carga los datos del archivo en un array de numpy
    data = np.loadtxt(file)
    data_list.append(data)
    

# Convierte la lista de arrays en un solo array de numpy.
# Nota: Esto funcionará si todos los arrays tienen la misma forma.
data_array = np.array(data_list)
x_test = data_array.transpose(1, 2, 0)

y_train = np.loadtxt("./UCIHARDataset/train/y_train.txt")
y_test = np.loadtxt("./UCIHARDataset/test/y_test.txt")  


print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

y_train = to_categorical(y_train - 1)
y_test = to_categorical(y_test - 1)

print(y_train.shape)
print(y_test.shape)

(7352, 128, 9)
(2947, 128, 9)
(7352,)
(2947,)
(7352, 6)
(2947, 6)


In [3]:
# Normalize x_train and x_test using the mean and standard deviation from x_train
x_train_mean = x_train.mean()
x_train_std = x_train.std()

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

print("x_train normalized: ", x_train.shape)
print("x_test normalized: ", x_test.shape)

x_train normalized:  (7352, 128, 9)
x_test normalized:  (2947, 128, 9)


In [9]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Input
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Input(shape=(128,9)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("Ejemplo con MLflow")



run = mlflow.start_run()
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test),callbacks=[mlflow.keras.MlflowCallback(run)])
mlflow.end_run()

Epoch 1/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5487 - loss: 1.1051 - val_accuracy: 0.8161 - val_loss: 0.5330
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8557 - loss: 0.3761 - val_accuracy: 0.8748 - val_loss: 0.3742
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9109 - loss: 0.2355 - val_accuracy: 0.8806 - val_loss: 0.3189
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9283 - loss: 0.1857 - val_accuracy: 0.9108 - val_loss: 0.2702
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9375 - loss: 0.1483 - val_accuracy: 0.9101 - val_loss: 0.3058
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9448 - loss: 0.1406 - val_accuracy: 0.9097 - val_loss: 0.3225
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9466 - loss: 0.1308 - val_accuracy: 0.9169 - val_loss: 0.3125
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9461 - loss: 0.1288 - val_accu